# Market Price

Helps build market price movements given individual trades

In [2]:
import pandas as pd
import os

def read_polymarket_csv(csv_filename):
    """
    Read a Polymarket trades CSV file and return a pandas DataFrame.

    Example usage:
    `$ df = read_polymarket_csv('polymarket_trades_0xd218e4_202505301025.csv')`
    
    Args:
        csv_filename (str): Name of the CSV file (e.g., 'polymarket_trades_0xd218e4_202505301025.csv')
    
    Returns:
        pandas.DataFrame: DataFrame with trade data, trade_dttm converted to datetime
    """
    try:
        # Construct full path to the CSV file in the data directory
        file_path = os.path.join("data", csv_filename)
        
        # Check if file exists
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Convert trade_dttm to datetime for proper analysis
        if 'trade_dttm' in df.columns:
            df['trade_dttm'] = pd.to_datetime(df['trade_dttm'])
        
        print(f"Successfully loaded {len(df)} trades from {csv_filename}")
        print(f"Columns: {list(df.columns)}")
        
        if 'trade_dttm' in df.columns:
            print(f"Date range: {df['trade_dttm'].min()} to {df['trade_dttm'].max()}")
        
        return df
        
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

def visualize_trade_prices(df, market_filter=None, time_window='1H'):
    """
    Visualize market price movements from individual trade data.
    
    Args:
        df (pd.DataFrame): Trade data with columns like 'trade_dttm', 'price', 'amount', 'side'
        market_filter (str): Optional filter for specific market
        time_window (str): Time window for aggregation ('1H', '1D', '15T' etc.)
    """
    
    # Filter data if specified
    if market_filter:
        df_filtered = df[df['market_title'] == market_filter].copy()
    else:
        df_filtered = df.copy()
    
    # Ensure datetime column
    df_filtered['trade_dttm'] = pd.to_datetime(df_filtered['trade_dttm'])
    
    # Sort by time
    df_filtered = df_filtered.sort_values('trade_dttm')
    
    plt.figure(figsize=(15, 12))
    
    # 1. SCATTER PLOT OF ALL TRADES
    plt.subplot(2, 2, 1)
    
    # Color trades by buy/sell
    buy_trades = df_filtered[df_filtered['side'] == 'buy']
    sell_trades = df_filtered[df_filtered['side'] == 'sell']
    
    plt.scatter(buy_trades['trade_dttm'], buy_trades['price'], 
               c='green', alpha=0.6, s=30, label='Buy Orders')
    plt.scatter(sell_trades['trade_dttm'], sell_trades['price'], 
               c='red', alpha=0.6, s=30, label='Sell Orders')
    
    plt.title('All Individual Trades')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.xticks(rotation=45)
    
    # 2. PRICE MOVEMENT LINE (LAST PRICE)
    plt.subplot(2, 2, 2)
    
    # Create time-based price series (using last price in each window)
    price_series = df_filtered.set_index('trade_dttm')['price'].resample(time_window).last().dropna()
    
    plt.plot(price_series.index, price_series.values, linewidth=2, color='blue')
    plt.title(f'Price Movement (Last Price per {time_window})')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.xticks(rotation=45)
    
    # 3. VOLUME-WEIGHTED AVERAGE PRICE (VWAP)
    plt.subplot(2, 2, 3)
    
    # Calculate VWAP for each time window
    df_resampled = df_filtered.set_index('trade_dttm').resample(time_window).agg({
        'price': 'mean',  # Average price
        'amount': 'sum'   # Total volume
    }).dropna()
    
    # Calculate VWAP
    df_resampled['vwap'] = (df_filtered.set_index('trade_dttm')['price'] * 
                           df_filtered.set_index('trade_dttm')['amount']).resample(time_window).sum() / df_resampled['amount']
    
    plt.plot(df_resampled.index, df_resampled['vwap'], linewidth=2, color='purple', label='VWAP')
    plt.plot(df_resampled.index, df_resampled['price'], linewidth=1, color='gray', alpha=0.7, label='Avg Price')
    
    plt.title(f'Volume-Weighted Average Price ({time_window})')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.xticks(rotation=45)
    
    # 4. CANDLESTICK-STYLE OHLC
    plt.subplot(2, 2, 4)
    
    # Create OHLC data
    ohlc_data = df_filtered.set_index('trade_dttm')['price'].resample(time_window).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    }).dropna()
    
    # Simple candlestick representation
    for i, (timestamp, row) in enumerate(ohlc_data.iterrows()):
        color = 'green' if row['close'] >= row['open'] else 'red'
        
        # High-low line
        plt.plot([timestamp, timestamp], [row['low'], row['high']], color='black', linewidth=1)
        
        # Open-close rectangle
        height = abs(row['close'] - row['open'])
        bottom = min(row['open'], row['close'])
        plt.bar(timestamp, height, bottom=bottom, color=color, alpha=0.7, width=pd.Timedelta(time_window)*0.8)
    
    plt.title(f'OHLC Candlesticks ({time_window})')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

def create_advanced_price_charts(df, market_filter=None):
    """
    Create more advanced price visualizations.
    """
    
    if market_filter:
        df_filtered = df[df['market_title'] == market_filter].copy()
    else:
        df_filtered = df.copy()
    
    df_filtered['trade_dttm'] = pd.to_datetime(df_filtered['trade_dttm'])
    df_filtered = df_filtered.sort_values('trade_dttm')
    
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # 1. PRICE WITH VOLUME
    ax1 = axes[0]
    ax1_vol = ax1.twinx()
    
    # Resample for cleaner visualization
    hourly_data = df_filtered.set_index('trade_dttm').resample('1H').agg({
        'price': 'last',
        'amount': 'sum'
    }).dropna()
    
    # Price line
    ax1.plot(hourly_data.index, hourly_data['price'], color='blue', linewidth=2, label='Price')
    ax1.set_ylabel('Price', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    
    # Volume bars
    ax1_vol.bar(hourly_data.index, hourly_data['amount'], alpha=0.3, color='gray', label='Volume')
    ax1_vol.set_ylabel('Volume', color='gray')
    ax1_vol.tick_params(axis='y', labelcolor='gray')
    
    ax1.set_title('Price and Volume Over Time')
    ax1.legend(loc='upper left')
    ax1_vol.legend(loc='upper right')
    
    # 2. BID-ASK SPREAD VISUALIZATION
    ax2 = axes[1]
    
    # Separate

In [ ]:
#df = read_polymarket_csv('poly_market_prices_10018_202505310659.csv')

df = read_polymarket_csv('poly_market_prices_16403_202505310709.csv')
data = df[df['question'] == 'Chargers vs. Texans - Chargers'].copy()

#calculate_gain_probability(data)

create_buy_results(data, 0.15, 'close_price', 'trade_dt')


df_sorted = data.sort_values('close_price').reset_index(drop=True)
    
range(len(df_sorted) - 1)

df